In [1]:
import pandas as pd

# Load the model output
df = pd.read_csv("data/churn_predictions.csv")

# Optional: check
df.head()

,customerID,churn_probability,value_segment,tenure,MonthlyCharges
0,7590-VHVEG,0.628532,Low Value,1,29.85
1,5575-GNVDE,0.151396,Mid Value,34,56.95
2,3668-QPYBK,0.474670,Mid Value,2,53.85
3,7795-CFOCW,0.062343,Low Value,45,42.30
4,9237-HQITU,0.828048,Mid Value,2,70.70


Apply Retention Policy Using Churn Probability

In [2]:
def retention_policy(row):
    if row["value_segment"] == "High Value" and row["tenure"] >= 25:
        return "Loyalty Bundle", 20, 0.70
    elif row["value_segment"] == "High Value":
        return "Contract Upgrade", 5, 0.25
    elif row["value_segment"] == "Mid Value":
        return "Support Call", 7, 0.55
    else:
        return "Discount", 10, 0.40

apply it to the dataframe

In [3]:
df[["retention_action", "retention_cost", "save_rate"]] = df.apply(
    lambda x: pd.Series(retention_policy(x)), axis=1
)

In [5]:
df["expected_saved_revenue"] = (
    df["churn_probability"] * df["MonthlyCharges"] * df["save_rate"]
)

df["retention_roi"] = df["expected_saved_revenue"] - df["retention_cost"]

# Optional flag for positive ROI
df["retain_flag"] = df["retention_roi"] > 0
df

,customerID,churn_probability,value_segment,tenure,MonthlyCharges,retention_action,retention_cost,save_rate,expected_saved_revenue,retention_roi,retain_flag
0,7590-VHVEG,0.628532,Low Value,1,29.85,Discount,10,0.40,7.504674,-2.495326,False
1,5575-GNVDE,0.151396,Mid Value,34,56.95,Support Call,7,0.55,4.742105,-2.257895,False
2,3668-QPYBK,0.474670,Mid Value,2,53.85,Support Call,7,0.55,14.058524,7.058524,True
3,7795-CFOCW,0.062343,Low Value,45,42.30,Discount,10,0.40,1.054847,-8.945153,False
4,9237-HQITU,0.828048,Mid Value,2,70.70,Support Call,7,0.55,32.198648,25.198648,True
...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,0.180843,High Value,24,84.80,Contract Upgrade,5,0.25,3.833876,-1.166124,False
7039,2234-XADUH,0.238170,High Value,72,103.20,Loyalty Bundle,20,0.70,17.205388,-2.794612,False
7040,4801-JZAZL,0.428157,Low Value,11,29.60,Discount,10,0.40,5.069384,-4.930616,False
7041,8361-LTMKD,0.765569,Mid Value,4,74.40,Support Call,7,0.55,31.327101,24.327101,True


Note: the higher the churn probability, the higher the expected revenue you can save exactly what business wants.

Rank Customers by ROI

In [6]:
priority_customers = df[df["retain_flag"]].sort_values(
    by="retention_roi", ascending=False
)

priority_customers.head(10)

,customerID,churn_probability,value_segment,tenure,MonthlyCharges,retention_action,retention_cost,save_rate,expected_saved_revenue,retention_roi,retain_flag
3615,6900-PXRMS,0.797509,High Value,26,105.75,Loyalty Bundle,20,0.7,59.035631,39.035631,True
4908,3957-LXOLK,0.789300,High Value,28,106.15,Loyalty Bundle,20,0.7,58.648905,38.648905,True
3980,8868-WOZGU,0.788308,High Value,28,105.70,Loyalty Bundle,20,0.7,58.326923,38.326923,True
4904,0114-PEGZZ,0.768662,High Value,33,107.55,Loyalty Bundle,20,0.7,57.868698,37.868698,True
5190,3001-UNBTL,0.779701,High Value,29,103.95,Loyalty Bundle,20,0.7,56.734935,36.734935,True
38,5380-WJKOV,0.760825,High Value,34,106.35,Loyalty Bundle,20,0.7,56.639646,36.639646,True
3642,2107-FBPTK,0.775254,High Value,30,104.10,Loyalty Bundle,20,0.7,56.492726,36.492726,True
574,6030-REHUX,0.721257,High Value,28,110.85,Loyalty Bundle,20,0.7,55.965913,35.965913,True
4645,5219-YIPTK,0.761171,High Value,33,104.40,Loyalty Bundle,20,0.7,55.626412,35.626412,True
4671,3393-FMZPV,0.790138,High Value,25,100.25,Loyalty Bundle,20,0.7,55.447917,35.447917,True


In [7]:
# Keep only top 10% customers by ROI
top_10_percent = int(0.10 * len(priority_customers))
top_customers = priority_customers.head(top_10_percent)

# Compute financial impact
total_saved = top_customers["expected_saved_revenue"].sum()
total_cost = top_customers["retention_cost"].sum()
net_savings = total_saved - total_cost

print(f"Top 10% retention simulation:")
print(f"Customers retained: {top_10_percent}")
print(f"Total expected savings: ${total_saved:,.2f}")
print(f"Total retention cost: ${total_cost:,.2f}")
print(f"Net savings: ${net_savings:,.2f}")

Top 10% retention simulation:
Customers retained: 330
Total expected savings: $13,605.77
Total retention cost: $3,844.00
Net savings: $9,761.77


In [9]:
# You already did this:
top_10_percent = int(0.10 * len(priority_customers))
top_customers = priority_customers.head(top_10_percent)
top_10_percent

330

In [10]:
top_customers[[
    "customerID",
    "value_segment",
    "tenure",
    "MonthlyCharges",
    "churn_probability",
    "retention_action",
    "retention_cost",
    "expected_saved_revenue",
    "retention_roi"
]]

,customerID,value_segment,tenure,MonthlyCharges,churn_probability,retention_action,retention_cost,expected_saved_revenue,retention_roi
3615,6900-PXRMS,High Value,26,105.75,0.797509,Loyalty Bundle,20,59.035631,39.035631
4908,3957-LXOLK,High Value,28,106.15,0.789300,Loyalty Bundle,20,58.648905,38.648905
3980,8868-WOZGU,High Value,28,105.70,0.788308,Loyalty Bundle,20,58.326923,38.326923
4904,0114-PEGZZ,High Value,33,107.55,0.768662,Loyalty Bundle,20,57.868698,37.868698
5190,3001-UNBTL,High Value,29,103.95,0.779701,Loyalty Bundle,20,56.734935,36.734935
...,...,...,...,...,...,...,...,...,...
3529,2880-FPNAE,Mid Value,2,74.20,0.834539,Support Call,7,34.057524,27.057524
5395,9095-HFAFX,Mid Value,5,81.00,0.764251,Support Call,7,34.047372,27.047372
6128,3317-VLGQT,Mid Value,14,78.10,0.792067,Support Call,7,34.023238,27.023238
6453,8634-MPHTR,High Value,47,100.05,0.671266,Loyalty Bundle,20,47.012139,27.012139


In [11]:
top_customers.head(10)  # see first 10 highest ROI customers

,customerID,churn_probability,value_segment,tenure,MonthlyCharges,retention_action,retention_cost,save_rate,expected_saved_revenue,retention_roi,retain_flag
3615,6900-PXRMS,0.797509,High Value,26,105.75,Loyalty Bundle,20,0.7,59.035631,39.035631,True
4908,3957-LXOLK,0.789300,High Value,28,106.15,Loyalty Bundle,20,0.7,58.648905,38.648905,True
3980,8868-WOZGU,0.788308,High Value,28,105.70,Loyalty Bundle,20,0.7,58.326923,38.326923,True
4904,0114-PEGZZ,0.768662,High Value,33,107.55,Loyalty Bundle,20,0.7,57.868698,37.868698,True
5190,3001-UNBTL,0.779701,High Value,29,103.95,Loyalty Bundle,20,0.7,56.734935,36.734935,True
38,5380-WJKOV,0.760825,High Value,34,106.35,Loyalty Bundle,20,0.7,56.639646,36.639646,True
3642,2107-FBPTK,0.775254,High Value,30,104.10,Loyalty Bundle,20,0.7,56.492726,36.492726,True
574,6030-REHUX,0.721257,High Value,28,110.85,Loyalty Bundle,20,0.7,55.965913,35.965913,True
4645,5219-YIPTK,0.761171,High Value,33,104.40,Loyalty Bundle,20,0.7,55.626412,35.626412,True
4671,3393-FMZPV,0.790138,High Value,25,100.25,Loyalty Bundle,20,0.7,55.447917,35.447917,True


These are the highest priority retention targets, with actions and net benefit clearly listed.

In [ ]:
top_customers.to_csv("data/top_10_percent_customers.csv", index=False)
print("Top 10% customers saved successfully!")